# ⚡ Homework 2: MapReduce Concepts & Spark Fundamentals
**MIS 769 - Big Data Analytics for Business | Spring 2026**

**Points:** 20 | **Due:** Sunday, February 8, 2026 @ 11pm Pacific

**Author:** Richard Young, Ph.D. | UNLV Lee Business School

**Compute:** CPU (free tier)

---

## What You'll Learn

1. Set up Apache Spark on Google Colab
2. Understand how Spark partitions data for parallel processing
3. **Measure and compare** processing performance with different configurations
4. Apply K-Means clustering and interpret business results
5. **Draw your own diagram** explaining distributed computing

---

## The Big Picture

When data gets too big for one computer, we split the work across many computers. **Spark** is the industry-standard tool for this.

```
YOUR LAPTOP                           SPARK CLUSTER
+----------------------+            +---------------------------+
|                      |            |     Driver Program        |
|   1 million rows     |            |         +----+            |
|   Takes: 10 minutes  |   ------>  |         |    |            |
|                      |            |    +----+----+----+       |
+----------------------+            |    |    |    |    |       |
                                    |   W1   W2   W3   W4       |
                                    |  250K 250K 250K 250K      |
                                    |  each worker in parallel  |
                                    +---------------------------+
```

---

## Part 1: Spark Environment Setup (3 points)

Let's install and configure Apache Spark on Google Colab.

In [ ]:
# Step 1: Install Java and PySpark
# IMPORTANT: Run this cell first, then wait for it to complete before running the next cell

import os
import subprocess

# Install Java (try Java 11 first, fallback to Java 8)
print("Installing Java...")
result = subprocess.run("apt-get update && apt-get install -y openjdk-11-jdk-headless",
                       shell=True, capture_output=True, text=True)

# Find Java installation path
java_paths = [
    "/usr/lib/jvm/java-11-openjdk-amd64",
    "/usr/lib/jvm/java-8-openjdk-amd64",
    "/usr/lib/jvm/default-java"
]

java_home = None
for path in java_paths:
    if os.path.exists(path):
        java_home = path
        break

if java_home:
    os.environ["JAVA_HOME"] = java_home
    os.environ["PATH"] = f"{java_home}/bin:" + os.environ.get("PATH", "")
    print(f"✅ JAVA_HOME set to: {java_home}")
else:
    print("⚠️ Java not found in expected paths")

# Verify Java works
result = subprocess.run("java -version", shell=True, capture_output=True, text=True)
print(f"Java version: {result.stderr.split(chr(10))[0]}")

# Install PySpark
!pip install pyspark --quiet
print("✅ PySpark installed!")

In [ ]:
# Step 2: Create a Spark Session with 2 cores
# Make sure Step 1 completed successfully before running this cell

import os

# Verify JAVA_HOME is set
if "JAVA_HOME" not in os.environ:
    # Try to find and set it
    for path in ["/usr/lib/jvm/java-11-openjdk-amd64", "/usr/lib/jvm/java-8-openjdk-amd64"]:
        if os.path.exists(path):
            os.environ["JAVA_HOME"] = path
            os.environ["PATH"] = f"{path}/bin:" + os.environ.get("PATH", "")
            break

print(f"JAVA_HOME: {os.environ.get('JAVA_HOME', 'NOT SET')}")

from pyspark.sql import SparkSession
import time

spark = (
    SparkSession.builder
    .appName("MIS769_HW2")
    .master("local[2]")  # Use 2 CPU cores
    .config("spark.driver.memory", "2g")  # Reduced for Colab compatibility
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)

print("✅ Spark Session Created!")
print(f"   App Name: {spark.sparkContext.appName}")
print(f"   Master: {spark.sparkContext.master}")
print(f"   Default Parallelism: {spark.sparkContext.defaultParallelism}")

**Question 1:** What does `local[2]` mean? What would `local[4]` do differently?

*Your answer here:*


---

## Part 2: Understanding Data Partitioning (5 points)

### 2.1 Create Sample Data and Partition It

In [ ]:
import random

# Create sample data: 100,000 random numbers
data_list = [random.randint(1, 1000) for _ in range(100000)]
print(f"Created {len(data_list):,} data points")

# Create RDD with 4 partitions
rdd = spark.sparkContext.parallelize(data_list, 4)
print(f"Number of partitions: {rdd.getNumPartitions()}")

In [ ]:
# Visualize how data is distributed across partitions
def count_partition(index, iterator):
    count = sum(1 for _ in iterator)
    yield (index, count)

partition_counts = rdd.mapPartitionsWithIndex(count_partition).collect()

print("\n📊 DATA DISTRIBUTION ACROSS PARTITIONS")
print("-" * 40)
for partition_id, count in partition_counts:
    bar = "█" * (count // 1000)
    print(f"Partition {partition_id}: {count:,} items {bar}")

### 2.2 The MapReduce Pattern

Let's see MapReduce in action: count how many times each number appears.

In [ ]:
# MapReduce: Count frequency of each number
# MAP: Transform each number to (number, 1)
# REDUCE: Sum up all the 1s for each number

start_time = time.time()

# The MapReduce pattern
frequency = (
    rdd
    .map(lambda x: (x, 1))           # MAP: (number, 1)
    .reduceByKey(lambda a, b: a + b)  # REDUCE: sum counts
)

# Get top 10 most frequent numbers
top_10 = frequency.takeOrdered(10, key=lambda x: -x[1])

elapsed = time.time() - start_time

print("📊 TOP 10 MOST FREQUENT NUMBERS")
print("-" * 30)
for num, count in top_10:
    print(f"   Number {num:4d}: {count:4d} times")
print(f"\n⏱️ Time taken: {elapsed:.3f} seconds")

---

## Part 3: Performance Experiment (5 points)

Let's measure how the number of cores affects processing speed.

In [ ]:
# Function to run our MapReduce with different core counts
def run_experiment(num_cores, data_size=500000):
    """Run MapReduce with specified number of cores and return timing."""

    # Stop existing session
    spark.stop()

    # Create new session with specified cores
    test_spark = (
        SparkSession.builder
        .appName(f"Test_{num_cores}_cores")
        .master(f"local[{num_cores}]")
        .config("spark.driver.memory", "4g")
        .getOrCreate()
    )

    # Create test data
    test_data = [random.randint(1, 10000) for _ in range(data_size)]
    test_rdd = test_spark.sparkContext.parallelize(test_data, num_cores * 2)

    # Time the MapReduce operation
    start = time.time()

    result = (
        test_rdd
        .map(lambda x: (x, 1))
        .reduceByKey(lambda a, b: a + b)
        .count()  # Force execution
    )

    elapsed = time.time() - start

    test_spark.stop()
    return elapsed

print("Running performance experiment...")
print("(This may take a minute)\n")

In [ ]:
# Run experiments with 1, 2, and 4 cores
import matplotlib.pyplot as plt

results = {}
for cores in [1, 2, 4]:
    print(f"Testing with {cores} core(s)...", end=" ")
    time_taken = run_experiment(cores)
    results[cores] = time_taken
    print(f"{time_taken:.3f} seconds")

# Recreate spark session for later use
spark = (
    SparkSession.builder
    .appName("MIS769_HW2")
    .master("local[2]")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

In [ ]:
# Visualize the results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

cores = list(results.keys())
times = list(results.values())

# Time comparison
axes[0].bar(cores, times, color=['#e74c3c', '#f39c12', '#27ae60'])
axes[0].set_xlabel('Number of Cores')
axes[0].set_ylabel('Time (seconds)')
axes[0].set_title('Processing Time by Core Count')
axes[0].set_xticks(cores)

# Speedup comparison
baseline = results[1]
speedups = [baseline / t for t in times]
axes[1].bar(cores, speedups, color=['#e74c3c', '#f39c12', '#27ae60'])
axes[1].axhline(y=1, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Number of Cores')
axes[1].set_ylabel('Speedup (x times faster)')
axes[1].set_title('Speedup Relative to 1 Core')
axes[1].set_xticks(cores)

plt.tight_layout()
plt.show()

print("\n📊 PERFORMANCE SUMMARY")
print("-" * 40)
for c, t in results.items():
    speedup = baseline / t
    print(f"{c} core(s): {t:.3f}s (speedup: {speedup:.2f}x)")

**Question 2:** Why doesn't 4 cores give exactly 4x speedup? What factors limit the speedup?

*Your answer here:*


---

## Part 4: Real Data Clustering with Spark ML (5 points)

### 4.1 Load a Real Dataset

In [ ]:
# Install datasets library
!pip install datasets -q

from datasets import load_dataset
import pandas as pd

# Load Netflix dataset from HuggingFace
print("Loading Netflix dataset...")
netflix_data = load_dataset("huggingfacejs/netflix-dataset", split="train")
df_pandas = netflix_data.to_pandas()

print(f"✅ Loaded {len(df_pandas):,} Netflix titles")
df_pandas.head()

In [ ]:
# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)

print(f"✅ Converted to Spark DataFrame")
print(f"   Partitions: {df_spark.rdd.getNumPartitions()}")
df_spark.printSchema()

### 4.2 Prepare Features for Clustering

In [ ]:
from pyspark.sql.functions import col, when, year, length
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans

# Create numeric features for clustering
df_features = df_spark.select(
    col("title"),
    col("type"),
    col("release_year").cast("int").alias("release_year"),
    length(col("description")).alias("description_length")
).dropna()

# Add binary feature for type
df_features = df_features.withColumn(
    "is_movie",
    when(col("type") == "Movie", 1).otherwise(0)
)

print(f"✅ Prepared {df_features.count():,} records for clustering")
df_features.show(5)

In [ ]:
# Assemble features into a vector
feature_cols = ["release_year", "description_length", "is_movie"]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw"
)

# Scale features
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

# Apply transformations
df_assembled = assembler.transform(df_features)
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

print("✅ Features assembled and scaled")

### 4.3 Run K-Means Clustering

In [ ]:
# Train K-Means model with 3 clusters
kmeans = KMeans(
    k=3,
    seed=42,
    featuresCol="features",
    predictionCol="cluster"
)

model = kmeans.fit(df_scaled)

# Get predictions
predictions = model.transform(df_scaled)

print("✅ K-Means clustering complete!")
print(f"   Number of clusters: {model.getK()}")

In [ ]:
# Analyze clusters
from pyspark.sql.functions import avg, count

cluster_stats = predictions.groupBy("cluster").agg(
    count("*").alias("count"),
    avg("release_year").alias("avg_year"),
    avg("description_length").alias("avg_desc_length"),
    avg("is_movie").alias("pct_movies")
).orderBy("cluster")

print("\n📊 CLUSTER ANALYSIS")
print("=" * 60)
cluster_stats.show()

In [ ]:
# Sample titles from each cluster
print("\n📺 SAMPLE TITLES FROM EACH CLUSTER")
print("=" * 60)

for cluster_id in range(3):
    print(f"\n--- Cluster {cluster_id} ---")
    samples = predictions.filter(col("cluster") == cluster_id).select("title", "type", "release_year").limit(5)
    samples.show(truncate=False)

**Question 3:** Based on your cluster analysis, what characterizes each cluster? Give each cluster a descriptive name (e.g., "Recent TV Shows", "Classic Movies", etc.)

*Your answer here:*

- Cluster 0: 
- Cluster 1: 
- Cluster 2: 


---

## Part 5: Draw Your Own Diagram (2 points)

Create a simple diagram (can be ASCII art, drawing, or description) showing how Spark processes your Netflix clustering job.

Include:
- How data is split across partitions
- What happens during the Map phase
- What happens during the Reduce/Aggregate phase

In [ ]:
# YOUR DIAGRAM HERE (as code comments, text, or create an image)

print("""
MY SPARK PROCESSING DIAGRAM:
============================

[Draw or describe your diagram here]

Example structure:

Netflix Data (8,800 titles)
        |
        v
   [PARTITION]
   /    |    \
  P1   P2    P3  (2,900 titles each)
  |    |     |
  v    v     v
 [MAP: Extract Features]
  |    |     |
  v    v     v
 [K-MEANS ITERATION]
  \    |    /
   \   |   /
    v  v  v
  [AGGREGATE: Update Centroids]
        |
        v
  3 Final Clusters

""")

---

## Clean Up

In [ ]:
# Stop Spark session
spark.stop()
print("✅ Spark session stopped. Notebook complete!")

---

## Submission Checklist

| Item | Points | Done? |
|------|--------|-------|
| Part 1: Spark setup complete | 3 | ☐ |
| Part 2: Data partitioning demonstrated | 5 | ☐ |
| Part 3: Performance experiment with analysis | 5 | ☐ |
| Part 4: K-Means clustering on Netflix data | 5 | ☐ |
| Part 5: Diagram explaining distributed processing | 2 | ☐ |
| **Total** | **20** | |

---

## Resources

- [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/)
- [How Netflix Uses Spark](https://netflixtechblog.com/spark-and-spark-streaming-at-netflix-21e9e5e3cd44)
- [K-Means Clustering Explained](https://scikit-learn.org/stable/modules/clustering.html#k-means)